In [ ]:
!pip install keras
!pip install numpy
!pip install pandas
!pip install tensorflow
!pip install lime
!pip install json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import shutil
import random
import numpy as np
import json
import zipfile
import os
import pandas as pd
import PIL
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
import os
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dense, Flatten
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import lime
from lime import lime_image
from PIL import Image

In [2]:
!wget "http://seppe.net/aa/assignment2/images.zip"
!7z x images.zip

--2023-05-15 21:36:37--  http://seppe.net/aa/assignment2/images.zip
Resolving seppe.net (seppe.net)... 37.139.1.16
Connecting to seppe.net (seppe.net)|37.139.1.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21223409265 (20G) [application/zip]
Saving to: ‘images.zip’

images.zip          100%[===================>]  19.77G  20.1MB/s    in 16m 44s 

2023-05-15 21:53:21 (20.2 MB/s) - ‘images.zip’ saved [21223409265/21223409265]


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 21223409265 bytes (20 GiB)

Extracting archive: images.zip
  3% 4096 Open              --
Path = images.zip
Type = zip
Physical Size = 21223409265
64-bit = +

  0%      0% 111 - images/1001046.jpg      

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
! rm images.zip

rm: cannot remove 'content/images.zip': No such file or directory


In [6]:
import os
images_path = '/content/images'
dataset_path = '/content/gdrive/My Drive/dataset.json'

Extract the price categories and store them in a library where they are linked to images. (We will later need this as a dataframe)

In [14]:
with open(dataset_path, "r") as f:
    data = json.load(f)
price_categories = {}
image_ids = []
for item in data:
    # Check if 'price_category' key is present in the 'item' dictionary
    if 'price_category' in item and item['price_category'] is not None:
        # Extract the price category
        price_category = item['price_category']['label']
        # Extract the image IDs from the "full_images" list and map them to their price category
        for img in item["more_details"]["full_images"]:
            image_id = img["image_id"]
            image_ids.append(image_id)
            price_categories[image_id] = price_category


Now I have a list of image id's and a library of price categories.
The file names of the images are just the id with .jpg behind it.
I want to create the training, validation and test set.
For this I need to make a pandas data frame with two columns, with the image file names and one with the categories.

First, we shuffle the image id's and then split them up in training, validation and test set lists of image id's.(The training, validation, test sizes are 70%, 20% and 10%. We could, however **change these values 0.7, 0.2, 0.1 to lower values if we want to use less of the data**.)

In [15]:
random.shuffle(image_ids)  # shuffle image id's

train_size = 0.09
val_size = 0 
test_size = 0.01

train_image_ids = image_ids[:int(len(image_ids) * train_size)]
val_image_ids = image_ids[int(len(image_ids) * train_size):int(len(image_ids) * (train_size + val_size))]
test_image_ids = image_ids[int(len(image_ids) * (train_size + val_size)):int(len(image_ids) * (train_size + val_size + test_size))]

Now we create lists of the filenames of these images.

In [16]:
train_filenames = [f"{image_id}.jpg" for image_id in train_image_ids]
val_filenames = [f"{image_id}.jpg" for image_id in val_image_ids]
test_filenames = [f"{image_id}.jpg" for image_id in test_image_ids]


Now, we create a pandas dataframe with two columns: "filename" and "price_category".

In [29]:
train_df = pd.DataFrame(
    {"id": train_filenames, "label": [price_categories[image_id] for image_id in train_image_ids]})
val_df = pd.DataFrame(
    {"id": val_filenames, "label": [price_categories[image_id] for image_id in val_image_ids]})
test_df = pd.DataFrame(
    {"id": test_filenames, "label": [price_categories[image_id] for image_id in test_image_ids]})

We set the paths for the training, validation, and test data directories and create the directories.

We set the paths for the training, validation, and test data directories and create the directories.

In [18]:
# Set the paths for the training, validation, and test data directories
train_data_dir = os.path.join(images_path, "train")
val_data_dir = os.path.join(images_path, "val")
test_data_dir = os.path.join(images_path, "test")

# Create the directories if they don't exist already
!mkdir -p "{train_data_dir}"
!mkdir -p "{val_data_dir}"
!mkdir -p "{test_data_dir}"

We move the image files to the appropriate directories.

In [19]:
for filename in train_filenames:
    src_path = os.path.join(images_path, filename)
    dst_path = os.path.join(train_data_dir, filename)
    shutil.move(src_path, dst_path)

for filename in val_filenames:
    src_path = os.path.join(images_path, filename)
    dst_path = os.path.join(val_data_dir, filename)
    shutil.move(src_path, dst_path)

for filename in test_filenames:
    src_path = os.path.join(images_path, filename)
    dst_path = os.path.join(test_data_dir, filename)
    shutil.move(src_path, dst_path)

Now we want to preprocess the data and then generate it.


We first define the image dimensions and the batch size. **(Change this, see what works best)**

In [20]:
img_width, img_height = 224, 224
batch_size = 32

We use ImageDataGenerator for preprocessing including rotation augmentation.

In [21]:
train_datagen = ImageDataGenerator(rescale=1. / 255, rotation_range=20, fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

We generate the training, validation, and testing datasets.

In [33]:
train_generator = train_datagen.flow_from_dataframe(
        dataframe = train_df,
        directory='/content/images/train',
        target_size=(224,224),
        x_col="id",
        y_col="label",
        subset="training",
        seed=42,
        shuffle=True,
        class_mode="categorical",
        batch_size=32)

validation_generator = train_datagen.flow_from_dataframe(
        dataframe = train_df,
        directory='/content/images/train',
        target_size=(224,224),
        x_col="id",
        y_col="label",
        subset="validation",
        seed=42,
        shuffle=True,
        class_mode="categorical",
        batch_size=32)

test_generator = test_datagen.flow_from_dataframe(
        dataframe = test_df,
        directory='/content/images/test',
        target_size=(224,224),
        x_col="id",
        y_col="label",
        seed=42,
        shuffle=False,
        class_mode="categorical",
        batch_size=32)

Found 6882 validated image filenames belonging to 4 classes.
Found 1214 validated image filenames belonging to 4 classes.
Found 105252 validated image filenames belonging to 4 classes.


Now we can finally build our Neural Network. We created one by hand at first, and then used a pretrained model (VGG16) to compare the results.

Define the CNN structure.

In [34]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

Compile the model.

In [35]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Train the model.(**Number of epochs can be changed**)

In [2]:
history = Model.fit_generator(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=5,
                              validation_data=val_generator, validation_steps=val_generator.samples // batch_size)

NameError: ignored

Evaluate the model.

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=test_generator.samples // batch_size)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)


Interpreter LIME


In [ ]:
explainer = lime_image.LimeImageExplainer()


def predict_wrapper(images):
    # This function is a wrapper around the model's prediction function
    # It takes in a batch of images (N, 224, 224, 3) and returns the predicted probabilities (N, 5)
    return model.predict(images)


# Get an example image from the validation set
example_image_path = os.path.join(val_data_dir, val_filenames[0])
example_image = Image.open(example_image_path)

# Explain the model's prediction for the example image
explanation = explainer.explain_instance(np.array(example_image), predict_wrapper, top_labels=5)

# Show the LIME visualization for the explanation
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
img_boundry = mark_boundaries(temp/2 + 0.5, mask)
Image.fromarray((img_boundry*255).astype(np.uint8))

Now, we do the same thing but with the pretrained model VGG16.

We first preprocess and generate.

In [22]:
# preprocessing according to VGG16
batch_size = 64
train_datagen = ImageDataGenerator(rescale=1. / 255, rotation_range=90,
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5,
                                     height_shift_range=0.5,
                                     horizontal_flip=True,
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(rescale=1. / 255,preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(rescale=1. / 255,preprocessing_function=preprocess_input)

# Generate the training, validation, and testing datasets (I followed the instructions on https://www.learndatasci.com/tutorials/hands-on-transfer-learning-keras/)
train_generator = train_datagen.flow_from_dataframe(
        dataframe = train_df,
        directory='/content/images/train',
        target_size=(224,224),
        x_col="id",
        y_col="label",
        subset="training",
        seed=42,
        shuffle=True,
        class_mode="categorical",
        batch_size=32)

validation_generator = train_datagen.flow_from_dataframe(
        dataframe = train_df,
        directory='/content/images/train',
        target_size=(224,224),
        x_col="id",
        y_col="label",
        subset="validation",
        seed=42,
        shuffle=True,
        class_mode="categorical",
        batch_size=32)

test_generator = test_datagen.flow_from_dataframe(
        dataframe = test_df,
        directory='/content/images/test',
        target_size=(224,224),
        x_col="id",
        y_col="label",
        seed=42,
        shuffle=False,
        class_mode="categorical",
        batch_size=32)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


Load without the top layer input_shape(img_width, img_height, 3) (the 3 is for the colors rbg).

In [ ]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in vgg_model.layers:
    layer.trainable = False
    # This is to make sure we don't train these layers

We add a new output layer with 4 categories. Then we create the new model with the pretrained layers and our new output layer.

In [ ]:
x = Flatten()(vgg_model.output)
output_layer = Dense(4, activation='softmax')(x)

model_VGG = Model(inputs=vgg_model.input, outputs=output_layer)

Compile the model.

In [ ]:
model_VGG.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Train the model. (**Number of epochs can be changed**)

In [ ]:
history = model_VGG.fit_generator(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=5,
                              validation_data=val_generator, validation_steps=val_generator.samples // batch_size)

Evaluate the model.


In [ ]:
test_loss, test_acc = model_VGG.evaluate_generator(test_generator, steps=test_generator.samples // batch_size)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

LIME interpreter

In [ ]:
explainer = lime_image.LimeImageExplainer()


def predict_wrapper_VGG(images):
    # This function is a wrapper around the model's prediction function
    # It takes in a batch of images (N, 224, 224, 3) and returns the predicted probabilities (N, 5)
    return model_VGG.predict(images)


# Get an example image from the validation set
example_image_path = os.path.join(val_data_dir, val_filenames[0])
example_image = Image.open(example_image_path)

# Explain the model's prediction for the example image
explanation = explainer.explain_instance(np.array(example_image), predict_wrapper_VGG, top_labels=5)

# Show the LIME visualization for the explanation
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
img_boundry = mark_boundaries(temp/2 + 0.5, mask)
Image.fromarray((img_boundry*255).astype(np.uint8))